We want to differentiate functions of the form
$$
R(s,t)=f(s-t)
$$
with
$$
f(v) = g\left(\frac{1}{2}v'Qv\right).
$$

The canonical example being the radial basis function (Gaussian covariance) with
$g(r)=e^{-r}$ and $Q=\Lambda^{-1}$ being the precision of the covariance (the inverse covariance matrix of the derivatives of  such a field).

$$
\begin{aligned}
\frac{\partial f}{\partial v_{i_1}}
&= g'\left(\frac{1}{2} v'Qv\right) \cdot \sum_{j_1} Q_{i_1j_1}v_{j_1} \\
&= g'\left(\frac{1}{2}v'Qv\right) \cdot \left(e_{i_1}'Qv\right)
\end{aligned}
$$

$$
\begin{aligned}
\frac{\partial^2 f}{\partial v_{i_1} \partial v_{i_2}}
&= g''\left(\frac{1}{2}v'Qv\right) \cdot \sum_{j_1,j_2} Q_{i_1j_1}v_{j_1}
       Q_{i_2j_2}v_{j_2} + \\
& \qquad g'\left(\frac{1}{2}v'Qv\right) \cdot Q_{i_1i_2} \\
&= g''\left(\frac{1}{2}v'Qv\right) \cdot \left(e_{i_1}'Qv \right) \left(e_{i_2}'Qv \right) + \\
& \qquad g'\left(\frac{1}{2}v'Qv\right) \cdot Q_{i_1i_2}
\end{aligned}
$$

$$
\begin{aligned}
\frac{\partial^3 f}{\partial v_{i_1} \partial v_{i_2}
                   \partial v_{i_3}}
&= g'''\left(\frac{1}{2}v'Qv\right) \cdot  \left(e_{i_1}'Qv \right) \left(e_{i_2}'Qv \right)
\left(e_{i_3}'Qv \right) + \\
& \qquad g''\left(\frac{1}{2}v'Qv\right) \left[Q_{i_1i_3} \left(e_{i_2}'Qv \right) + Q_{i_2i_3} \left(e_{i_1}'Qv \right)\right] \\
& \qquad g''\left(\frac{1}{2}v'Qv\right) \cdot Q_{i_1i_2} \left(e_{i_3}'Qv \right) \\
&= g'''\left(\frac{1}{2}v'Qv\right) \cdot  \left(e_{i_1}'Qv \right) \left(e_{i_2}'Qv \right)
\left(e_{i_3}'Qv \right) + \\
& \qquad g''\left(\frac{1}{2}v'Qv\right) \left[Q_{i_1i_3} \left(e_{i_2}'Qv \right) + Q_{i_2i_3} \left(e_{i_1}'Qv \right) + Q_{i_1i_2} \left(e_{i_3}'Qv \right) \right] \\
\end{aligned}
$$

$$
\begin{aligned}
\frac{\partial^4 f}{\partial v_{i_1} \partial v_{i_2}
                   \partial v_{i_3} \partial v_{i_4}}
&= g''''\left(\frac{1}{2}v'Qv\right) \left(e_{i_1}'Qv \right) \left(e_{i_2}'Qv \right)
\left(e_{i_3}'Qv \right) \left(e_{i_4}'Qv \right) + \\
& \qquad g'''\left(\frac{1}{2}v'Qv\right) \cdot \biggl[Q_{i_1i_4} \left(e_{i_2}'Qv \right) \left(e_{i_3}'Qv \right) + 
Q_{i_2i_4} \left(e_{i_1}'Qv \right) \left(e_{i_3}'Qv \right) +
Q_{i_3i_4} \left(e_{i_1}'Qv \right) \left(e_{i_2}'Qv \right) +
Q_{i_1i_3} \left(e_{i_2}'Qv \right) \left(e_{i_4}'Qv \right)
+ Q_{i_2i_3} \left(e_{i_1}'Qv \right) \left(e_{i_4}'Qv\right) + Q_{i_1i_2} \left(e_{i_3}'Qv \right) \left(e_{i_4}'Qv\right) \biggr] + \\
& \qquad g''\left(\frac{1}{2}v'Qv\right) \left[Q_{i_1i_3}Q_{i_2i_4} + Q_{i_2i_3}Q_{i_1i_4} + Q_{i_1i_2}Q_{i_3i_4}\right]
\end{aligned}
$$

In [ ]:
import numpy as np
from dataclasses import dataclass

@dataclass
class isofunc(object):
    
    Q : np.ndarray
                 
    def func(self, arg, order=0):
        raise NotImplementedError('must implement up to 4th order derivative')
        
    def deriv0(self, v):
        # v_flat of shape (-1,Q.shape[0])
        d = self.Q.shape[0]
        v_flat = v.reshape((-1,d))
        Qv = np.einsum('ij,jk->ik', v_flat, self.Q)
        arg = (Qv * v_flat).sum(-1)
        return self.func(arg/2, order=0).reshape(v.shape[:-1])

    def deriv1(self, v):
        # v_flat of shape (-1,Q.shape[0])
        d = self.Q.shape[0]
        v_flat = v.reshape((-1,d))
        Qv = np.einsum('ij,jk->ik', v_flat, self.Q)
        arg = (Qv * v_flat).sum(-1)
        g_1 = self.func(arg/2, order=1)
        return (g_1[:,None] * Qv).reshape(v.shape[:-1] + (d,))

    def deriv2(self, v):
        # v_flat of shape (-1,Q.shape[0])
        d = self.Q.shape[0]
        v_flat = v.reshape((-1,d))
        Qv = np.einsum('ij,jk->ik', v_flat, self.Q)
        arg = (Qv * v_flat).sum(-1)
        g_1 = self.func(arg/2, order=1)
        g_2 = self.func(arg/2, order=2)
        
        V_2 = np.einsum('i,ij,ik->ijk', g_2, Qv, Qv)
        V_1 = np.einsum('i,jk->ijk', g_1, self.Q)
        
        return (V_1 + V_2).reshape(v.shape[:-1] + (d, d))
    
    def deriv3(self, v):
        # v_flat of shape (-1,Q.shape[0])
        d = self.Q.shape[0]
        v_flat = v.reshape((-1,d))
        Qv = np.einsum('ij,jk->ik', v_flat, self.Q)
        arg = (Qv * v_flat).sum(-1)
        g_2 = self.func(arg/2, order=2)
        g_3 = self.func(arg/2, order=3)
        
        V_3 = np.einsum('i,ij,ik,il->ijkl', g_3, Qv, Qv, Qv)
        V_2 = (np.einsum('i,ij,kl->ijkl', g_2, Qv, Q) +
               np.einsum('i,ik,jl->ijkl', g_2, Qv, Q) +
               np.einsum('i,il,jk->ijkl', g_2, Qv, Q))
        return (V_3 + V_2).reshape(v.shape[:-1] + (d,)*3)

    def deriv4(self, v):
        # v_flat of shape (-1,Q.shape[0])
        d = self.Q.shape[0]
        v_flat = v.reshape((-1,d))
        Qv = np.einsum('ij,jk->ik', v_flat, self.Q)
        arg = (Qv * v_flat).sum(-1)
        
        g_2 = self.func(arg/2, order=2)
        g_3 = self.func(arg/2, order=3)
        g_4 = self.func(arg/2, order=4)
        
        V_4 = np.einsum('i,ij,ik,il,im->ijklm', g_4, Qv, Qv, Qv, Qv)
        
        V_3 = (np.einsum('i,ij,ik,lm->ijklm', g_3, Qv, Qv, Q) +
               np.einsum('i,ij,il,km->ijklm', g_3, Qv, Qv, Q) +
               np.einsum('i,ij,im,kl->ijklm', g_3, Qv, Qv, Q) +
               np.einsum('i,ik,il,jm->ijklm', g_3, Qv, Qv, Q) +
               np.einsum('i,ik,im,jl->ijklm', g_3, Qv, Qv, Q) +
               np.einsum('i,il,im,jk->ijklm', g_3, Qv, Qv, Q))
        
        V_2 = (np.einsum('i,jk,lm->ijklm', g_2, Q, Q) +
               np.einsum('i,jl,km->ijklm', g_2, Q, Q) +
               np.einsum('i,jm,kl->ijklm', g_2, Q, Q))
        
        return (V_4 + V_3 + V_2).reshape(v.shape[:-1] + (d,)*4)

class gaussian(isofunc):
    
    def func(self, arg, order=0):
        return (-1)**order * np.exp(-arg)
    

                 

In [ ]:
rng = np.random.default_rng(0)
X = rng.standard_normal((5,3))
Q = X.T @ X / 5
G = gaussian(Q)
V = rng.standard_normal((10,3))
W = rng.standard_normal((5,3))
G.deriv0(V)

In [ ]:
np.exp(-(V * (V @ Q)).sum(1)/2)

In [ ]:
G.deriv1(V)[0]

In [ ]:
(-np.exp(-(V[0] * (Q @ V[0])).sum()/2)*(Q @ V[0]))

In [ ]:
G.deriv2(V)[0]

In [ ]:
np.exp(-(V[0] * (Q @ V[0])).sum()/2) * (np.multiply.outer(Q @ V[0], Q @ V[0]) - Q)

In [ ]:
G.deriv3(V)[0]

In [ ]:
G.deriv4(V)[0]

In [ ]:
from gplasso.kernels import covariance_structure
xval = np.linspace(0, 10, 21)
yval = np.linspace(0, 10, 21)
zval = np.linspace(0, 10, 21)

grid = np.meshgrid(xval, yval, zval, indexing='ij')

K = covariance_structure.gaussian(precision=Q, var=1, grid=grid)

In [ ]:
C00 = K.C00(V, W)

In [ ]:
A = np.array([np.subtract.outer(V[:,i], W[:,i]) for i in range(V.shape[-1])])
A = np.transpose(A, [1, 2, 0])
np.linalg.norm(G.deriv0(A) - C00) / np.linalg.norm(C00)

In [ ]:
C10 = K.C10(V, W)
C10.shape

In [ ]:
np.linalg.norm(G.deriv1(A)[:,:,0]-C10[:,:,0]) / np.linalg.norm(C10[:,:,0])

In [ ]:
np.linalg.norm(G.deriv1(A)-C10) / np.linalg.norm(C10)

In [ ]:
G.deriv2(A).shape

In [ ]:
C20 = K.C20(V, W)
C20.shape

In [ ]:
np.linalg.norm(G.deriv2(A)[:,:,0,0]-C20[:,:,0,0]) / np.linalg.norm(C20[:,:,0,0])

In [ ]:
np.linalg.norm(G.deriv2(A)-C20) / np.linalg.norm(C20)

In [ ]:
C21 = K.C21(V, W)
C21.shape

In [ ]:
np.linalg.norm(G.deriv3(A)[:,:,0,0,0]+C21[:,:,0,0,0]) / np.linalg.norm(C21[:,:,0,0,0])

In [ ]:
np.linalg.norm(G.deriv3(A)+C21) / np.linalg.norm(C21)

In [ ]:
G.deriv3(A).shape

In [ ]:
C22 = K.C22(V, W)

In [ ]:
np.linalg.norm(G.deriv4(A)-C22) / np.linalg.norm(C22)